# Cleaning functions

## 1.Functions to correct the categorical variables

### 1.1 Gender

In [13]:
def correct_gender(data):
    """
    Receive the dataframe and replace in the gender column 1 with m and 2 with f
    
    Parameters
    ----------
    df: pandas.core.frame.DataFrame
           
    Returns
    -------
    pandas.core.series.Series: 
        Returns the modified dataframe
    """
    data['gender'] = data.gender.map({1: 'm', 2:'f'})
    return data

### 1.2 Cholesterol, Gluc, Smoke, Alco, Active

In [14]:
# Funcion que convierte el valor de las variables categoricas en categorias
def correct_categorical(df):
    """
    Receive a dataframe, iterate over the categorical variables and map them to their respective categories
    
    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        
    Returns
    -------
    pandas.core.series.Series: 
        Returns the modified dataframe
    """
    info = { 'cholesterol': {1:'normal', 2:'limitrofe', 3:'alto'},
                'gluc': {1:'normal', 2: 'prediabetes', 3: 'diabetes'},
                'smoke':{1:'si', 0:'no'},
                'alco':{1:'si', 0:'no'},
                'active':{1:'si', 0:'no'}}
    for col, mapping in info.items():
        df[col]= df[col].map(mapping)
        
    return df

## 2.Functions to correct the ontinuous variables

### 2.1 Blood pressure cleaning function (Ver de hacer mas simples las fx y de poner una cita)

In [15]:
def correct_TA_PP(data)   :
    """
        Corrige los valores que se consideran errores de carga de la TA sistólica y diastólica.
        Imputa los datos faltantes entre TAS y TAD con un delta de 40mmHg.
        Se genera la columna PP (presión de pulso)   
    """
    def correct_TA(row, value_ranges,step):
        """
            Recibe un valor y un rango, corrige el valor en una forma recursiva con diferentes estrategias.
            Si el valor corregido no se encuentra en el rango devuelve nan
        ------
            Parameters:
                data pandas DataFrame
            Return:
                data pandas DataFrame
        """
        if step == 1: row = abs(row)
        elif step == 2: row = row*10
        elif step == 3: row = row/100
        if (row >= value_ranges[0] and row <= value_ranges[1]):
            return row
        elif step==4:
            return 999
        else: 
            return correct_TA(row,value_ranges, step+1)
    
    def complete_TA(row):
        """
            Completa el valor de TAS o TAD con un delta de 40 en relación con el otro parámetro.
            Si no lo puede calcular devuelve nan.
        """
        if row['TAS']==999 or row['TAD']==999:  # si TAD o TAD  no tienen datos
            if row['TAS']==999 and row['TAD']==999:  # Si ambos estan sin datos completa con nan
                import numpy as np
                row['TAS'] = np.NaN
                row['TAD'] = np.NaN
            else:
                if row['TAS']==999:         # Si TAS sin datos 
                    row['TAS'] = row['TAD']+40
                else:
                    row['TAD'] = row['TAS']-40  # si TAD sin datos
                    if row['TAD'] >= 120:
                        row['TAD'] = 120
        return row

    def control_TA(row):
        """
            Chequea que la TAS sea mayor que la TAD, si no es asi reemplaza la TAD por TAS-40
        """
        if not (row['TAS'] > row['TAD']):
            row['TAD'] = row['TAS'] -40
        return row

    # Corrección TAS y TAD (se consideran diferentes errores de carga)
    data['TAS'] = data["ap_hi"].apply(correct_TA, args=((90,240),0))
    data['TAD'] = data.ap_lo.apply(correct_TA, args=((40,120),0))
    # Se eliminan las columnas que no se utilizarán
    data.drop(columns=['ap_hi','ap_lo'], inplace=True)
    # Se imputan los datos faltantes de TAD o TAS utilizando un delta de 40 mmHg entre uno y otro
    data = data.apply(complete_TA, axis=1)
    # Se corrigen las incoherencias entre TAS y TAD (considerando la TAS como valor de referencia)
    data = data.apply(control_TA, axis=1)
    # Se eliminan valores nulos
    data = data.dropna()
    # Se genera la variable PP (presión de pulso)
    data['PP'] = data.TAS - data.TAD
    # Se resetea el indice
    data.reset_index(drop=True, inplace=True)
    return data

### 2.2 Age

In [16]:
def correct_age(data):
    """
    Remove age-related columns that will not be used. 
    Creates the "AgeCat" column that contains the discretized age in years. 
    
    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        
    Returns
    -------
    pandas.core.series.Series: 
        Returns the modified dataframe
    """
    # Remove age-related columns that will not be used.
    data.drop(columns=['age','AgeGroup'], inplace=True)  
    # Creates the "AgeCat" column that contains the discretized age in years.
    bins = [0, 30, 40, 50, 60, 70]
    data['AgeCat'] = pd.cut(data.AgeinYr, bins, right = False)
    return data

### 2.3 Height

In [17]:
def eliminar_outliers(df, columna, lim_inf, lim_sup):
    """
    Eliminate outliers based on the selected limits.
    
    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        DataFrame to parse
    arg_2: str
        Column to parse
    lim_inf: int
        Lower limit 
    lim_sup: int
        Upper limit  
        
    Returns
    -------
    pandas.core.series.Series
        DataFrame with outliers removed
    """
    print("Shape of the initial dataframe:", df.shape)
    out_data = df[(df[columna] > lim_inf) & (df[columna]< lim_sup)]
    print("Shape of the dataframe after removing outliers:", out_data.shape)
    out_data.reset_index(drop=True, inplace=True)
    return out_data

### 2.4 Weight (Hay cita??)

In [18]:
def correct_weight(data):
    """
    Corrects weights less than 50 kg based on the theoretical weight according to height
    
    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        
    Returns
    -------
    pandas.core.series.Series: 
        Returns the modified dataframe
    """
    data['new_weight'] = data.apply(lambda x: x['height'] - 100 if x['weight'] < 50 else x['weight'], axis = 1)
    data['weight'] = data['new_weight']
    data.drop(columns='new_weight', inplace=True)
    
    return data

### 2.5 BMI

In [19]:
def create_BMI(data):
    """
    Based on the weight and height, it calculates the BMI, replacing the "BMI" column of the original dataset.
    Also it creates a column with categorized BMI, replacing the "BMICat" column.
        
    Parameters
    ----------
    data: pandas.core.frame.DataFrame
        DataFrame to parse
        
    Returns
    -------
    pandas.core.series.DataFrame: 
         Returns the modified dataframe
    """
    # Se eliminan las columnas BMI y BMI cat
    data.drop(columns=['BMI', 'BMICat'], inplace=True)
    # Se crean las columnas bmi y bmi_cat
    data['bmi'] = ((data['weight'] / (data['height'] ** 2)) * 10000).round(2)
    data['bmi_cat'] = pd.cut(data.bmi, bins=[0, 18.5, 24.9, 29.9, 70], labels=['Bajo peso', 'Peso normal', 'Sobrepeso', 'Obesidad'])
    return data

## Function for scanning individual columns in a standardized way

In [20]:
def exploracion(df, columna):
    """
    Realiza una descripción y un boxplot de la columna seleccionada.
    
    Parámetros
    ----------
    df: pandas.core.frame.DataFrame
        DataFrame a analizar
    columna: str
        Columna a analizar
           
    Returns
    -------
    pandas.core.series.Series: 
        Descripción de la columna seleccionada
        
    matplotlib.axes._subplots.AxesSubplot: 
        Boxplot de la columna seleccionada
    """
    describe = df[columna].describe()
    grafico = sns.boxplot(data= df, x = columna)
    titulo = ("Distribucion de la columna: " + columna.title())
    plt.title(titulo)
    plt.show()

    print("Descripcion de la columna", columna.title(), "\n", describe)

#### Funciones para calcular el score de riesgo de Framingham y el Europeo

In [21]:
#Framingham Risk Score for Women

def framingham_women(row):
    """
        Calcula el score de Framingham para mujeres
        Devuelve el score como un entero
    """
    import random as rd
    points=0
    # Puntaje por edad
    if  row['AgeinYr'] <=34: points -= 7
    elif row['AgeinYr'] <=39: points -= 3
    elif row['AgeinYr'] <=44: points -= 0
    elif row['AgeinYr'] <=49: points += 3
    elif row['AgeinYr'] <=54: points += 6
    elif row['AgeinYr'] <=59: points +=8
    elif row['AgeinYr'] <=64: points +=10
    elif row['AgeinYr'] <=69: points +=12
    elif row['AgeinYr'] <=74: points +=14
    else: points +=16
    
    # Puntaje por niveles de colesterol total y tabaquismo
    if row['AgeinYr'] <=39:
        if row['cholesterol'] =='normal':  points += rd.choice([0,4]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 8
        else: points += rd.choice([11,13]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 9
    elif row['AgeinYr'] <=49:
        if row['cholesterol'] =='normal':  points += rd.choice([0,3]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 6
        else: points += rd.choice([8,10]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 7
    elif row['AgeinYr'] <=59:
        if row['cholesterol'] =='normal':  points += rd.choice([0,2]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 4
        else: points += rd.choice([5,7]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 4
    elif row['AgeinYr'] <=69:
        if row['cholesterol'] =='normal':  points += rd.choice([0,1]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 2
        else: points += rd.choice([3,4]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 2
    else:
        if row['cholesterol'] =='normal':  points += rd.choice([0,2]) # hay 2 subca tegorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 1
        else: points += 2
        if row['smoke'] =='si': points += 1

    # Puntaje según nivel de colesterol HDL
    #Hace un choices entre -1, 1 y 2 con los weights puestos segun active, bmi_cat y gluc
    prob_HDL = [0.33,0.34,0.33]
    if row['bmi_cat']=='Obesidad' or row['bmi_cat']== 'Sobrepeso': 
        prob_HDL[0] -= 0.15; prob_HDL[1] += 0.05; prob_HDL[2] += 0.10
    if row['active']=='no': 
        prob_HDL[0] -= 0.10; prob_HDL[1] += 0.04; prob_HDL[2] += 0.06
    if row['gluc'] == 'diabetes': 
        prob_HDL[0] -= 0.08; prob_HDL[1] += 0.04; prob_HDL[2] += 0.04
    points += rd.choices([-1,1,2], weights=prob_HDL)[0]

    # Puntaje por niveles de TAS
    if row['TAS'] <= 120: points += rd.choice([0,1])    # Discrimina entre valores de TAS con y sin tratamiento
    elif row['TAS'] <= 129: points += rd.choice([1,3])  # como no tenemos esa información se selecciona al azar
    elif row['TAS'] <=139: points += rd.choice([2,4])
    elif row['TAS'] <=159: points += rd.choice([3,5])
    else: points += rd.choice([4,6])
    
    return points


In [22]:
#Framingham Risk Score for Men 
def framingham_men(row):
    """
        Calcula el score de Framingham para hombres
        Devuelve el score como un entero
    """
    import random as rd
    points=0
    # Puntaje por edad
    if  row['AgeinYr'] <=34: points -= 9
    elif row['AgeinYr'] <=39: points -= 4
    elif row['AgeinYr'] <=44: points -= 0
    elif row['AgeinYr'] <=49: points += 3
    elif row['AgeinYr'] <=54: points += 6
    elif row['AgeinYr'] <=59: points +=8
    elif row['AgeinYr'] <=64: points +=10
    elif row['AgeinYr'] <=69: points +=11
    elif row['AgeinYr'] <=74: points +=12
    else: points +=13

    # Puntaje por niveles de colesterol total y tabaquismo
    if row['AgeinYr'] <=39:
        if row['cholesterol'] =='normal':  points += rd.choice([0,4]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 7
        else: points += rd.choice([9,1]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 8
    elif row['AgeinYr'] <=49:
        if row['cholesterol'] =='normal':  points += rd.choice([0,3]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 5
        else: points += rd.choice([6,8]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 5
    elif row['AgeinYr'] <=59:
        if row['cholesterol'] =='normal':  points += rd.choice([0,2]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 3
        else: points += rd.choice([4,5]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 3
    elif row['AgeinYr'] <=69:
        if row['cholesterol'] =='normal':  points += rd.choice([0,1]) # hay 2 subcategorias que no puedo definir 
        if row['cholesterol'] =='limitrofe': points += 1
        else: points += rd.choice([2,3]) # hay 2 subcategorias que no puedo definir 
        if row['smoke'] =='si': points += 1
    elif row['cholesterol'] =='alto': 
        points += 1
        if row['smoke'] =='si': points += 1

    # Puntaje según nivel de colesterol HDL
    #Hace un choices entre -1, 1 y 2 con los weights puestos segun active, bmi_cat y gluc
    prob_HDL = [0.25,0.25,0.25,0.25]
    if row['bmi_cat']=='Obesidad' or row['bmi_cat']== 'Sobrepeso': 
        prob_HDL[0] -= 0.10;prob_HDL[1] -= 0.10; prob_HDL[2] += 0.1; prob_HDL[3] += 0.1
    if row['active']=='no': 
        prob_HDL[0] -= 0.10;prob_HDL[1] -= 0.05; prob_HDL[2] += 0.05; prob_HDL[3] += 0.1
    if row['gluc'] == 'diabetes': 
        prob_HDL[0] -= 0.05;prob_HDL[1] -= 0.05; prob_HDL[2] += 0.05; prob_HDL[3] += 0.05
    points += rd.choices([-1,0,1,2], weights=prob_HDL)[0]

    # Puntaje por niveles de TAS
    if row['TAS'] <= 129: points += rd.choice([0,1])      # Discrimina entre valores de TAS con y sin tratamiento
    elif row['TAS'] <=139: points += rd.choice([1,2])           # como no tenemos esa información se selecciona al azar
    elif row['TAS'] <=159: points += rd.choice([1,2])
    else: points += rd.choice([2,3])
    
    return points


In [23]:
def create_table_ESC():
  """
    Crea un array multidimensional con los datos de la tabla de riesgo cardiovascular de la ESC
    Devuelve el array de numpy
  """
  esc_table = [
              [[
                  [[7,8,9,10,12],
                  [5,5,6,7,8],
                  [3,3,4,5,6],
                  [2,2,3,3,4]],

                  [[4,4,5,6,7],
                  [33,3,4,5],
                  [2,2,2,3,3],
                  [1,1,2,2]],

                  [[2,2,3,3,4],
                  [1,2,2,2,3],
                  [1,1,1,1,2],
                  [1,1,1,1,1]],

                  [[1,1,1,2,2],
                  [1,1,1,1,1],
                  [0,1,1,1,1],
                  [0,0,1,1,1]],
                  
                  [[0,0,0,0,0],
                  [0,0,0,0,0],
                  [0,0,0,0,0],
                  [0,0,0,0,0]]
                  ],
                [
                  [[13,15,17,19,22],
                  [9,10,12,13,16],
                  [6,7,8,9,11],
                  [4,5,5,6,7]],
                  [[8,9,10,11,13],
                  [5,6,7,8,9],
                  [3,4,5,5,6],
                  [2,3,3,4,4]],
                  [[4,5,5,6,7],
                  [3,3,4,4,5],
                  [2,2,2,3,3],
                  [1,1,2,2,2]],
                  [[2,2,3,3,4],
                  [1,2,2,2,3],
                  [1,1,1,1,2],
                  [1,1,1,1,1]],
                  [[0,0,0,1,1],
                  [0,0,0,0,0],
                  [0,0,0,0,0],
                  [0,0,0,0,0]]
                  ]
              ],
              [[
                  [[14,16,19,22,26],
                  [9,11,13,15,16],
                  [6,8,9,11,13],
                  [4,5,6,7,9]],
                  [[9,11,13,15,18],
                  [76,9,10,12],
                  [4,5,6,7,9],
                  [3,3,4,5,5]],
                  [[6,7,8,10,12],
                  [4,5,6,7,8],
                  [3,3,4,5,6],
                  [2,2,3,3,4]],
                  [[4,4,5,6,7],
                  [2,3,3,4,5],
                  [2,2,2,3,3],
                  [1,1,2,2,2]],
                  [[1,1,1,2,2],
                  [1,1,1,1,1],
                  [0,1,1,1,1],
                  [0,0,1,1,1]]
                  ],
                [
                  [[26,30,35,41,47],
                  [18,21,25,29,34],
                  [13,15,17,20,24],
                  [9,10,12,14,17]],
                  [[18,21,24,28,33],
                  [12,14,17,20,24],
                  [8,10,12,14,17],
                  [6,7,8,10,12]],
                  [[12,13,16,19,22],
                  [8,9,11,13,16],
                  [5,6,8,9,11],
                  [4,4,5,6,8]],
                  [[7,8,10,12,14],
                  [5,6,7,8,10],
                  [3,4,5,6,7],
                  [2,3,3,4,5]],
                  [[2,2,3,3,4],
                  [1,2,2,2,3],
                  [1,1,1,2,2],
                  [1,1,1,1,1]]
                  ]
              ]
            ]
  return  np.array(esc_table, dtype='object')

In [24]:
def esc_score(row,table):
    """
        Recibe una fila y la tabla de la ESC y sobre ella calcula el riesgo cardiovascular
        Devuelve el score correspondiente como entero
    """
    # Posicion 0 es gender: 0 fem y 1 masc
    # Posicion 1 es smoker: 0 es no smoker y 1 es smoker
    # Posicion 2 es edad: 4 menos de 40, 3 menos de 50, 2 menos de 60, 1 menos de 70, 0 mas de 70
    # Posicion 3 es nivel de TAS:3 es menos 120, 2 menos 140, 1 menos de 160, 0 mas 160
    # Posicion 4 es nivel de colesterol:0 y 1 menos de 200, 2 entre 200-240 y 3 y 4 mas de 240

    import random as rd
    # Calcula las posiciones en al array
    if row['gender'] == 'f': p1=0
    else: p1=1

    if row['smoke'] =='no': p2=0
    else: p2=1

    if row['AgeinYr'] <= 40: p3= 4
    elif row['AgeinYr'] <= 50: p3= 3
    elif row['AgeinYr'] <= 60: p3= 2
    else: p3= 0

    if row['TAS'] <= 120: p4= 3
    elif row['TAS'] <= 140: p4= 2
    elif row['TAS'] <= 160: p4= 1
    else: p4= 0

    if row['cholesterol'] == 'normal': p5 = rd.choice([0,1])
    elif row['cholesterol'] == 'limitrofe': p5 = 2
    else: p5 = rd.choice([3,4])
    
    # Retorna el puntaje según la tabla
    return table[p1,p2,p3,p4][p5]

